In [ ]:
%matplotlib inline
import json 
import matplotlib.pyplot as plt
import os
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import keras 

# import numpy
# import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf


from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix


from keras import backend as K

# Main configs

In [2]:
loadFromMerged=True
loadFromIndexes= False
Mapper='S'
IgnoreEmpty= True
FoldID ="1"
Epoch_count=100
Batch_size=5000

# Load the data the old way


In [3]:
# x_data= [] 
# y_data= [] 


# with open( '../files/txt/seq_mapping_large.txt' ) as f:
#     x_data =   f.readlines()

# with open( '../files/txt/command_mapping_large.txt' ) as f:
#     y_data = f.readlines()
    
    
# x_data =[ np.array([ int(y) for y in x.strip().split( ' ') ])   for x in  x_data ] 
# y_data =[ x.strip().split(' ') for x in  y_data ] 


#  Load The Data The New Way

In [4]:
#  mapps the input records to a integer array for the input
def mapping_x( inp, includeDirection = False , TrimAt= 15 ):
    if includeDirection:
        return np.array([ int(x["packet_length"]) * (1 if x['packet_source']=='hub' else -1)  for x in inp ][:15])
    else:
        return np.array([ int(x["packet_length"])  for x in inp ][:15])

In [5]:
def mapping_y_service(inp):
    return np.array(  list(set([x["event"] for x in inp])) if (len(inp )>0) else ["none"] )

def mapping_y_service_event(inp):
    return np.array(  list(set([ "%s-%s"%( x["event"] ,x["val"] ) for x in inp])) if (len(inp )>0) else ["none"] )

def mapping_y_device_service(inp):
    return np.array(  list(set([ "%s & %s"%( x["device"] ,x["event"] ) for x in inp])) if (len(inp )>0) else ["none"] )

def mapping_y_full(inp):
    return np.array(  list(set([ "%s & %s & %s"%( x["device"] ,x["event"], x['val'] ) for x in inp])) if (len(inp )>0) else ["none"] )


In [6]:
# this cleans the data removing emply nodes and turning the nodes into sarrays by calling the mapping function 
def clean_data( x_data, y_data , removeempty=True, Mapping='S'):
    cleans = [] 
    cleans = (sorted([ x for x in y_data if (removeempty and len(y_data[x]) > 0) or not removeempty  ] ))
    
    ret_x  = [x_data[x] for x in cleans]
    ret_y  = [y_data[x] for x in cleans] 
    
    print( len(y_data), len(cleans) )
    
    ret_x  = [ mapping_x(x) for x in ret_x ] 
    ret_y_s = [ mapping_y_service(y) for y in ret_y ]
    if Mapping=='S':
        ret_y  = [ mapping_y_service(y) for y in ret_y ]
    elif Mapping=='SE':
        ret_y  = [ mapping_y_service_event(y) for y in ret_y ]
    elif Mapping=='DS':
        ret_y  = [ mapping_y_device_service(y) for y in ret_y ]
    elif Mapping=='F':
        ret_y  = [ mapping_y_full(y) for y in ret_y ]
    return ret_x, ret_y, ret_y_s

In [7]:
x= []
y= []

x_test = []
y_test = []
y_test_service= []

x_train = {}
y_train = {}

test_names = []

add_to_trainig = [0,2]

if loadFromMerged:
    print("in load from merged")
    with open(  '../files/train/test/test_homes/final_upload/Merged_final_with_home/merged_hub_segments_final.json'  ) as f:
        y_data = json.load(f)

    with open(  '../files/train/test/test_homes/final_upload/Merged_final_with_home/merged_pcap_segments_final.json'  ) as f:
        x_data = json.load(f)
        
#     with open(  '../files/train/merged/hub_segments_2.json'  ) as f:
#         y_data = json.load(f)

#     with open(  '../files/train/merged/pcap_segments_2.json'  ) as f:
#         x_data = json.load(f)
  
    if len( y_data ) != len(x_data) :
        print( pick )
        
    
    x_train,y_train, y_train_service= clean_data( x_data, y_data, IgnoreEmpty , Mapping=Mapper )
    
    #     continue
#     if loadFromIndexes:
#         print("load from indexes")
#         with open("../files/train/merged/items_2_test-train_indexes.json")  as f:
#             index_info = json.load(f)


#         for i in index_info[FoldID]["test"]:
#             x_test[str(i)]=(x_data[str(i)] )
#             y_test[str(i)]=(y_data[str(i)] )

#         for i in index_info[FoldID]["train"]:
#             x_train[str(i)]=(  x_data[str(i)] )
#             y_train[str(i)]=(  y_data[str(i)] )
        
#         x_test_t,y_test_t= clean_data( x_test, y_test, IgnoreEmpty , Mapping=Mapper)
#         x_test.append(x_test_t)
#         y_test.append(y_test_t)
    #     else :
    print("loading from test files")
    test_files = sorted(glob.glob( '../files/train/test/test_homes/final_upload/usecases/pcap_segments_final_final/home*.json' ))
    print( "found files : " , len(test_files) )
    for pick  in test_files:
        fname  = os.path.basename(pick)
        test_names.append( fname )
        with open( os.path.join( '../files/train/test/test_homes/final_upload/usecases/hub_segments_final_final/', fname) ) as f:
            y_data_test = json.load(f)

        with open( os.path.join('../files/train/test/test_homes/final_upload/usecases/pcap_segments_final_final/', fname) ) as f:
            x_data_test = json.load(f)


        t_x,t_y, t_z= clean_data( x_data_test, y_data_test, False , Mapping=Mapper )

#         if test_files.index(pick) in add_to_trainig:
#             x_test_t,y_test_t, y_test_service_t= clean_data( x_data_test, y_data_test, IgnoreEmpty , Mapping=Mapper)
#             x_train.extend(x_test_t)
#             y_train.extend(y_test_t)
#             y_train_service.extend(y_test_service_t)


        x_test.append(t_x)
        y_test.append(t_y)
        y_test_service.append(t_z)
            
#     x_test = x_data[ index_info["1"]["test"]  ]
#     y_test = y_data[ index_info["1"]["test"]  ]
    
#     x_train = x_data[ index_info["1"]["train"]  ]
#     y_train = y_data[ index_info["1"]["train"]  ]
#     x.extend(t_x)
#     y.extend(t_y)
else:
    for pick in sorted(glob.glob( '../files/train/hub_segments/*.json' )):
        fname  = os.path.basename(pick)
        test_names.append( fname )
        with open( os.path.join( '../files/train/hub_segments/', fname) ) as f:
            y_data = json.load(f)

        with open( os.path.join('../files/train/pcap_segments/', fname) ) as f:
            x_data = json.load(f)

        if len( y_data ) != len(x_data) :
            print( pick )
            continue

        t_x,t_y= clean_data( x_data, y_data, True )

        x.extend( t_x)
        y.extend(t_y)

x= np.array(x)
y= np.array(y)

# x_train = np.append( x_train, x_test[0] , axis=0)
# x_train = np.append( x_train, x_test[2] , axis=0)

# y_train = np.append( y_train, y_test[0] , axis=0)
# y_train = np.append( y_train, y_test[2] , axis=0)


len(x_train), len(y_test)

in load from merged
58958 57867
loading from test files
found files :  3
32069 32069
19968 19968
6404 6404


(57867, 3)

In [ ]:
len(y_data)

In [ ]:
list(np.unique(  np.concatenate( y_train  )))

In [13]:
classes = sorted(list(np.unique(  np.concatenate( y_train  ))))
print([ (i , classes[i]) for i in range( len(classes) ) ])

service_classes = sorted(list(np.unique(  np.concatenate( y_train_service  ))))
[ (i , service_classes[i]) for i in range( len(service_classes) ) ]

[(0, 'acceleration'), (1, 'activity'), (2, 'battery'), (3, 'button'), (4, 'colorTemperature'), (5, 'contact'), (6, 'level'), (7, 'lock'), (8, 'motion'), (9, 'ping'), (10, 'status'), (11, 'switch'), (12, 'temperature'), (13, 'threeAxis'), (14, 'unknown'), (15, 'water')]


[(0, 'acceleration'),
 (1, 'activity'),
 (2, 'battery'),
 (3, 'button'),
 (4, 'colorTemperature'),
 (5, 'contact'),
 (6, 'level'),
 (7, 'lock'),
 (8, 'motion'),
 (9, 'ping'),
 (10, 'status'),
 (11, 'switch'),
 (12, 'temperature'),
 (13, 'threeAxis'),
 (14, 'unknown'),
 (15, 'water')]

In [ ]:
[ (i , classes[i]) for i in range( len(classes) ) ]

In [ ]:
def is_clean_event( inp, return_clean= True  ):
    if return_clean:
        return  'no_logs' not in inp and 'lock-unlocked' not in inp and 'on/off-XXX' not in inp and 'raw-XXX' not in inp and 'read_attr_-_raw-XXX' not in inp
    else:
        return  'lock-locked' in inp or 'lock-unlocked'  in inp or 'on/off-XXX' in inp or  'raw-XXX' in inp  or 'read_attr_-_raw-XXX' in inp 
    
def is_clean_service( inp, return_clean= True  ):
    
    if return_clean:
        return  'no_logs' not in inp and 'unknown' not in inp and 'read_attr_-_raw' not in inp #and 'ping' not in inp 
    else:
        return  'no_logs' in inp or  'unknown' in inp  or 'read_attr_-_raw' in inp #or 'ping' in inp 
#     return  "contact-open" not in inp and 'contact-closed' not in inp

In [ ]:

y_test[0][0]

In [ ]:
toKeep = [ i for i in range(len(y_train)) if is_clean_event( y_train[i]) ] if Mapper=='SE' else [ i for i in range(len(y_train)) if is_clean_service( y_train[i]) ]
x_train= [ x_train[i] for i in toKeep ]
y_train= [ y_train[i] for i in toKeep ]


In [ ]:
for j in range(len(x_test)):
    toChange= [ i for i in range(len(y_test[j])) if is_clean_service( y_test[j][i], False) ]
    y_test[j] = [ (y_test[j][i] if i not in toChange else np.array( ['none'])) for i in range(len(y_test[j])) ]

In [ ]:
# classes.remove('read_attr_-_raw-XXX')
# classes.remove('on/off-XXX')
# classes.remove('raw-XXX')
# classes.remove('lock-unlocked')
# classes.remove('lock-locked')


# classes.remove('read_attr_-_raw')
# classes.remove('on/off')
# classes.remove('raw')
classes.remove('unknown')

# classes.remove('lock')
# # classes.remove('lock')


# classes.remove('switch-on')



service_classes= ["","",""]


In [8]:
def pre_process_raw( x_data,y_data, dim_size = 128, zero_pad = False, normalize = False ,classes=None, twoD= False ):
#  y data 
    if classes is None:
        classes  = sorted(list(np.unique(  np.concatenate( y_data  ))))
    else :
        classes = sorted(classes)
    y_data_categorical = []  

    for x in y_data:
        temp = np.zeros( len(classes) )
        for y in x : 
            if y in classes:
                temp[ classes.index( y ) ] = 1
        y_data_categorical.append( temp )
    y_data_categorical = np.vstack(y_data_categorical)

#     x_data = np.array( x_data) / 1500.0
    
    x_data_temp = [] 
    
    if not zero_pad:
        if twoD:
            for x in x_data:
                temp = [] #list(x)
                lst = list(x)
                while dim_size**2 - len(temp )   > len(lst):
                    temp.extend(lst)

                while len(temp) < dim_size**2:
                    temp.append( 0 )

                x_data_temp.append(np.array( temp).reshape(dim_size,dim_size))


            x_data_temp = np.array( x_data_temp )
            x_data_temp=x_data_temp.reshape(x_data_temp.shape+(1,))
        else: 
            temp = [] 
            lst = list(x)
            for x in x_data:
                temp = [] #list(x)
                lst = list(x)
                while dim_size - len(temp )   > len(lst):
                    temp.extend(lst)

                while len(temp) < dim_size:
                    temp.append( 0 )
                
                x_data_temp.append(np.array( temp))
            
    else :
        x_data_temp = sequence.pad_sequences(x_data, maxlen=dim_size)
    
    
    if normalize:
        x_data_temp = np.array( x_data_temp) / 1500.0
    else :
        x_data_temp = np.array(x_data_temp)
    
    
    return x_data_temp ,y_data_categorical , classes

# Model Evaluation Methods

In [33]:
def make_recall_shit( inp ):
    tp = inp[1][1]
    tn = inp[0][0]
    fp = inp[0][1] 
    fn = inp[1][0]
    
    acc = (tp+tn)*1.0 / ( tp+tn+fp+fn)*1.0
    recall = tp*1.0/ ( tp+fn ) *1.0
    prec = tp*1.0 / ( tp+fp )*1.0
    
#     F= 2.0*( prec* recall )/ (prec+recall)
    F= 2.0*( tp)/ (2*tp + fp + fn)
    
    return acc, recall, prec, F

def acc_match( true, pred ):
    """
    returns exact mathc accuracy
    """
 
    return (len( [ x  for x  in  [np.sum(np.abs( true[i]- pred[i] )) for i in range(len(true))] if x  == 0]))*1.0 / len(true)


# def acc_none_zero ( true, pred ):
    

def acc_match_wierd( true, pred ):
    """
    returns exact mathc accuracy
    """
    level = 6 
    switch = 11
    threeAxis=13
    accel = 0 
    status=10
    contact=5
    
    counter  = 0 
    for i in range( len (true) ):
        if np.sum(np.abs( true[i]- pred[i] ))==0 :
            counter+=1
        else : 
            t_rec = np.array(list( pred[i]))
            
            if true[i][level]==1 and true[i][switch]==1 and t_rec[level]==1 :
                t_rec[switch]=1
            
            if true[i][threeAxis]==1 and true[i][accel]==1 and t_rec[threeAxis]==1:
                t_rec[accel] =1
            
            if true[i][status]==1 and true[i][contact]==1 and t_rec[status]==1:
                t_rec[contact]=1
#             print(t_rec , true[i])    
            if np.sum(np.abs( true[i]- t_rec ))==0 :
                counter+=1   
            
             
            
    
    return counter*1.0 / len(true)


def print_info(y_test, pred , classes , confidance=0.5 ):
    
    counts = np.sum( y_test.astype(int) , axis=0)
    
    pred[pred>=confidance] = 1
    pred[pred<confidance] = 0
    
#     acc_wierd  =acc_match_wierd(y_test, pred)
    
    conf= multilabel_confusion_matrix( y_test , pred.astype(int), labels= range(len(classes)))
    accs = [make_recall_shit(x) for x in conf]
    print( "%30s  %8s   %8s  %8s  %8s %8s %16s"  %( "Class","Accuracy", "Recall","Precision","F Score" , "Count", "TP/TN/FP/FN"))
    print( "------------------------------------------------------------------------" )
    
    for index in range(len(classes)):
        tp = conf[index][1][1]
        tn = conf[index][0][0]
        fp = conf[index][0][1] 
        fn = conf[index][1][0]
        print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %d/%d/%d/%d"  %
             (classes[index],
              accs[index][0],
              accs[index][1],
              accs[index][2],
              accs[index][3],
              counts[index],
                  tp ,
                tn ,
                fp ,
                fn ))
    n_zeros_true = len([ x  for x  in  [np.sum(np.abs( y_test[i] )) for i in range(len(y_test))] if x  == 0]  )
    n_zeros_pred = len([ x  for x  in  [np.sum(np.abs( pred[i] )) for i in range(len(pred))] if x  == 0]  )
    
    accs = np.nan_to_num(accs)
    
    print ("------------------------------------------------------------------------")
    print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %d/%d/%d/%d"  %
             ("AVERAGES",
              np.average( accs, axis=0)[0],
              np.average( accs, axis=0)[1],
              np.average( accs, axis=0)[2],
              np.average( accs, axis=0)[3],
              len(y_test),
                  0 ,
                0,
                0 ,
                0 ))
    
    print ( "Exact Match ACC : %.5f " % acc_match( y_test, pred )  )
#     print ( "Wierd Exact Match ACC : %.5f" % acc_wierd)
    print ( "Total Records : %d " % len(y_test)  )
    print ( "Total ZXeros in True : %d (%.3f)%%" % (n_zeros_true ,  n_zeros_true * 1.0/ len(y_test)  ))
    print ( "Total ZXeros in Test : %d (%.3f)%%" % (n_zeros_pred ,  n_zeros_pred * 1.0/ len(y_test)  ) )
    print ('=============================================================================')
    

In [10]:
def make_readable_results ( inp , classes , conffidance=True):
    ret = [] 
    inp =inp.astype(int)
    for xx in range(len(inp)) :
        u = inp[xx]
        temp = []
        for j in range(len(u)) : 
            if u[j] >0:
                temp.append(classes[j])
        ret.append(temp)
    return ret


def makeReadable( model , data, gt, path , classes, x, confidance=0.7):
    pred_temp = model.predict(data)
    print_info(gt, pred_temp, classes , confidance=confidance)
    print( len(classes ), len( pred_temp[0] ) )
    xcc= make_readable_results(pred_temp , classes)
    y_gt = make_readable_results( gt, classes )
    temp_dic = {} 
    for pick in range(len(xcc)): 
        temp_dic[ pick +1 ] =  { 'seq': str(data[pick]),
                               'pred': xcc[pick],
                                'true':y_gt[pick]
                               }   

    with open(path , 'w') as f:
        json.dump(temp_dic , f, indent=4)


# def makeReadable( model , data, gt, path , classes, confidance=0.7, x):
#     pred_temp = model.predict( data)
#     print_info(gt, pred_temp, classes , confidance=confidance)
#     xcc= make_readable_results( pred_temp , classes )
#     temp_dic = {} 
#     for pick in range(len(xcc)): 
#         temp_dic[ pick +1 ] = xcc[pick]  

#     with open(path , 'w') as f:
#         json.dump(temp_dic , f, indent=4)

# Create the Model 

# Calcualte per class accuracy

# Random Forest baseline calc

In [ ]:
dim_size= 50
x_random_forest_train,y_random_forest_train, _ = pre_process_raw( x_train, y_train , dim_size, zero_pad=True, normalize=False, classes=classes)
# x_random_forest_test,y_random_forest_test, _ = pre_process_raw( x_test[0], y_test[0] , dim_size, zero_pad=True, normalize=False, classes=classes)
rf_tests  = [ pre_process_raw( x_test[i], y_test[i] , dim_size, zero_pad=True, normalize=False, classes=classes) for i in range(len(x_test)) ] 
# x,y, classes = pre_process_raw( x_data, y_data , dim_size, zero_pad=True, normalize=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=960, max_depth=9050,
                             random_state=0 )
t_hist = clf.fit(x_random_forest_train, y_random_forest_train)




# print(clf.feature_importances_)

# print(clf.predict([[0, 0, 0, 0]]))
# from sklearn import metrics
# scores = cross_val_score(clf, x_random_forest_train, y_random_forest_train, cv=10, scoring='accuracy')

In [ ]:
for i in range(len(rf_tests)) :
    print( "==================HOME Case : %s =============" % test_names[ i] )
    rf_pred= clf.predict( rf_tests[i][0])
    print_info( rf_tests[i][1], rf_pred, classes)

In [ ]:
# makeReadable( data=rf_tests[0][0], model=clf, classes=classes, confidance=0.7,gt=rf_tests[0][1], path='sk_home_out.json' )

In [ ]:
# print_info(y_random_forest_test, rf_pred, classes)

In [ ]:
# print_info(y_random_forest_test, rf_pred, classes)

In [ ]:
# print ( "mean : %f \nstd: %f\nmax:%f" %( scores.mean(), scores.std(), scores.max()) )

# LSTM 

In [ ]:
# cnts  = np.unique(np.array([ len(x) for x  in x_train ]), return_counts=True)
# # np.sort( cnts[1] )
# cnts

In [ ]:
test_names

In [14]:
# create the model
dim_size =15
x_lstm_prossed_train,y_lstm_prossed_train, _ = pre_process_raw( x_train, y_train , dim_size, zero_pad=False, normalize=False,classes=classes)
_, y_s_lstm_processed_train ,_ =  pre_process_raw( x_train, y_train_service , dim_size, zero_pad=False, normalize=False,classes=service_classes)
# x_lstm_prossed_test,y_lstm_prossed_test, _ = pre_process_raw( x_test, y_test_2 , dim_size, zero_pad=True, normalize=False,classes=classes)
lstm_tests  = [ pre_process_raw( x_test[i], y_test[i] , dim_size, zero_pad=False, normalize=False, classes=classes) for i in range(len(x_test)) ] 
lstm_tests_services  = [ pre_process_raw( x_test[i], y_test_service[i] , dim_size, zero_pad=False, normalize=False, classes=service_classes) for i in range(len(x_test)) ] 



In [15]:
x_lstm_prossed_train[0]

array([290,  99, 412,  99, 290,  99, 412,  99, 290,  99, 412,  99,   0,
         0,   0])

In [16]:
for i in range( len(lstm_tests) ):
    print( len( lstm_tests[i][0] ), len( lstm_tests_services[i][0] ) )

32069 32069
19968 19968
6404 6404


In [17]:
# x_lstm_prossed_test2 = np.expand_dims(x_lstm_prossed_test,axis=1)
# x_lstm_prossed_train2 =np.expand_dims(x_lstm_prossed_train,axis=1)

for tt  in range( len(lstm_tests) ):
    lstm_tests[tt]= (lstm_tests[tt][0].reshape(len(lstm_tests[tt][0]),dim_size,1) , lstm_tests[tt][1],lstm_tests_services[tt][1] )
# x_lstm_prossed_test2 = x_lstm_prossed_test.reshape(len(x_lstm_prossed_test),dim_size,1)
x_lstm_prossed_train2 =x_lstm_prossed_train.reshape(len(x_lstm_prossed_train),dim_size,1)

# y_lstm_prossed_test2 = y_lstm_prossed_test.reshape(len(y_lstm_prossed_test),len(classes),1)
# y_lstm_prossed_train2 =y_lstm_prossed_train.reshape(len(y_lstm_prossed_train),len(classes),1)

In [18]:
np.sum(y_s_lstm_processed_train)

60562.0

In [20]:
## Bidirectional LSTM :D 
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Flatten


import numpy as np
import os
import tensorflow as tf
from keras.models import Model

from keras.layers import Input, merge, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, Dropout, Conv2DTranspose, UpSampling2D, Lambda
from keras.layers import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization as bn
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import RMSprop
from keras import regularizers
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers.merge import add
from keras.models import load_model
import numpy as np
from keras.regularizers import l2
from keras.layers import Conv1D
from keras.layers.merge import add
from keras.layers import Dense,Conv1D,Dropout,Activation,BatchNormalization,MaxPooling1D,Flatten,Masking,TimeDistributed
from keras.layers.recurrent import LSTM,GRU,SimpleRNN
from keras.models import Input,Sequential,Model
from keras.layers.merge import add
from keras.optimizers import Adam
from keras.losses import MSE,MSLE

model2 = Sequential()


def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss


import tensorflow as tf

def f1_perRow(y_true, y_pred):
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)


def f1_perClass(y_true, y_pred):

    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=1)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=1)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=1)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss_perClass(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=1)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=1)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=1)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return (1 - K.mean(f1))**2

def f1_loss_perRow(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return (1 - K.mean(f1))**2



weights =np.array([1.0/(50.0/50358.0),
1.0/(52.0/50358.0),
1.0/(3.0/50358.0),
1.0/(5.0/50358.0),
1.0/(4.0/50358.0),
1.0/(11.0/50358.0),
1.0/(58.0/50358.0),
1.0/(210.0/50358.0),
1.0/(376.0/50358.0),
1.0/(118.0/50358.0),
1.0/(121.0/50358.0),
1.0/(694.0/50358.0),
1.0/(230.0/50358.0),
1.0/(221.0/50358.0),
1.0/(105.0/50358.0),
1.0/(98.0/50358.0),
1.0/(16560.0/50358.0),
1.0/(30490.0/50358.0),
1.0/(94.0/50358.0),
1.0/(96.0/50358.0),
1.0/(452.0/50358.0),
1.0/(68.0/50358.0),
1.0/(50.0/50358.0),
1.0/(45.0/50358.0),])
# weights = weights/ np.max(weights)

# model2.add(LSTM(50, input_shape=(dim_size, 1), return_sequences=True))
# model2.add(TimeDistributed(Dense(1, activation='relu')))
# model2.add(Dropout(0.2))
# model2.add(Flatten())
# model2.add(Dense(127, activation='relu'))
# model2.add(Dense(len(classes), activation='sigmoid'))
# # model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])


# # model2.add(Embedding(word_count, dim_size, input_length=dim_size))
# # model2.add(Dropout(0.1))

inputs  = Input( ( dim_size,1 ) )

# bi_d_1 = Bidirectional( LSTM(200 ,  recurrent_dropout=0.12, return_sequences=True),input_shape=[dim_size,1],merge_mode='concat') (inputs)
lstm_1 =  LSTM(60 ,  recurrent_dropout=0.04, return_sequences=True)(inputs)
# lstm_2 = LSTM(30 ,  recurrent_dropout=0.14, return_sequences=True)(lstm_1)

bi_d_1 =Dense(128, activation='relu')  (lstm_1)
lstm_1 =  Dense(128, activation='relu')(bi_d_1)
lstm_2 = Dense(128, activation='relu')(lstm_1)



# td_1    = TimeDistributed(Dense(256, activation='relu'))(lstm_2)
# dout_1  = Dropout(0.1)(td_1)
dout_1  = Dropout(0.1)(lstm_2)
flt_1   = Flatten()(dout_1)
dense_1 = Dense(128, activation='relu')(flt_1)
dout_2  = Dropout(0.2)(dense_1)






out = Conv1D(128,3,padding='same')(inputs)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = Dropout(0.2)(out)
out = Conv1D(128,3,padding='same')(out)
# out = Flatten()(out)
# out = MaxPooling1D(2,padding='same', name ='pooling')(out)


out = Conv1D(128,3,padding='same')(out)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = Dropout(0.2)(out)
out = Conv1D(128,3,padding='same')(out)
# out = Flatten()(out)
# out = MaxPooling1D(2,padding='same', name ='pooling2')(out)


out = Conv1D(128,3,padding='same')(out)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = Dropout(0.2)(out)
out = Conv1D(128,3,padding='same')(out)
# out = Flatten()(out)
# out = MaxPooling1D(2,padding='same', name ='pooling')(out)



# fl_out_1 = Flatten()(dout_2)

fl_out_cnn = Flatten()(out)

# out_new = concatenate( [fl_out_1, fl_out_cnn] , name='mergerguy')
out_new = concatenate( [dout_2, fl_out_cnn] , name='mergerguy')

dens_out_1 = Dense( 128, activation='relu' )(out_new)
dens_out_2 = Dense( 128, activation='relu' )(dens_out_1)
dens_out_3 = Dense( 128, activation='relu' )(dens_out_2)

# fl2  = Flatten()(out_new)

out_put_final = Dense(len(classes), activation='sigmoid', name='Event_output')(dens_out_3)

toService_1 = Dense( 130, name="to_service1" )(dens_out_3)
toService_1 = Dense( 130, name="to_service2" )(toService_1)

service_output = Dense(len(classes  ), activation="sigmoid", name = 'service_output')(toService_1)



model2 = Model(inputs=[inputs], outputs=[service_output])




# model2.add(Bidirectional( LSTM(200 ,  recurrent_dropout=0.12, return_sequences=True),input_shape=[dim_size,1],merge_mode='concat'))
# model2.add( LSTM(60 ,  recurrent_dropout=0.04, return_sequences=True))
# model2.add( LSTM(30 ,  recurrent_dropout=0.14, return_sequences=True))
# # model2.add(Bidirectional( LSTM(100 ,  recurrent_dropout=0.04, return_sequences=True),merge_mode='concat'))
# model2.add(TimeDistributed(Dense(256, activation='relu')))
# model2.add(Dropout(0.1))
# model2.add(Flatten())
# model2.add(Dense(128, activation='relu'))
# model2.add(Dropout(0.2))
# model2.add(Dense(len(classes), activation='sigmoid'))

# model2.compile(loss=weighted_categorical_crossentropy(weights=weights), optimizer='adam', metrics=['accuracy'])



losses = {
#     "service_output": f1_loss_perClass ,
    "service_output": f1_loss_perRow ,
    "service_output": "categorical_crossentropy",
}
lossWeights = {#"service_output": 20,
               "service_output": 30.0 ,
    "service_output": 20}
 


model2.compile(loss=losses,loss_weights=lossWeights, optimizer='adam', metrics=[f1_perRow,f1_perClass,'acc'])
# model2.compile(loss=losses, loss_weights=lossWeights, optimizer='adam', metrics=['accuracy'])

checkpoint = ModelCheckpoint('IoTDownNet', monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

print(model2.summary())
hist2 = model2.fit(x_lstm_prossed_train2, y_lstm_prossed_train, epochs=100, batch_size=7500, shuffle=True, callbacks=callbacks_list)





____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 15, 1)         0                                            
____________________________________________________________________________________________________
conv1d_7 (Conv1D)                (None, 15, 128)       512         input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_4 (BatchNorm (None, 15, 128)       512         conv1d_7[0][0]                   
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 15, 128)       0           batch_normalization_4[0][0]      
___________________________________________________________________________________________

57867/57867 [==============================] - 1s - loss: 18.2062 - f1_perRow: 0.3459 - f1_perClass: 0.6794 - acc: 0.4668     
Epoch 10/100
57867/57867 [==============================] - 1s - loss: 17.8389 - f1_perRow: 0.3517 - f1_perClass: 0.6852 - acc: 0.4668     
Epoch 11/100
57867/57867 [==============================] - 1s - loss: 17.5457 - f1_perRow: 0.3581 - f1_perClass: 0.6893 - acc: 0.4668     
Epoch 12/100
57867/57867 [==============================] - 1s - loss: 17.5431 - f1_perRow: 0.3574 - f1_perClass: 0.6960 - acc: 0.4666     
Epoch 13/100
57867/57867 [==============================] - 1s - loss: 17.0502 - f1_perRow: 0.3675 - f1_perClass: 0.6996 - acc: 0.4665     
Epoch 14/100
57867/57867 [==============================] - 1s - loss: 16.7822 - f1_perRow: 0.3742 - f1_perClass: 0.7050 - acc: 0.4666     
Epoch 15/100
57867/57867 [==============================] - 1s - loss: 16.5824 - f1_perRow: 0.3799 - f1_perClass: 0.7061 - acc: 0.4665     
Epoch 16/100
57867/57867 [=======

57867/57867 [==============================] - 1s - loss: 15.4967 - f1_perRow: 0.4371 - f1_perClass: 0.7172 - acc: 0.4967     
Epoch 68/100
57867/57867 [==============================] - 1s - loss: 15.3494 - f1_perRow: 0.4369 - f1_perClass: 0.7183 - acc: 0.5214     
Epoch 69/100
57867/57867 [==============================] - 1s - loss: 15.3190 - f1_perRow: 0.4185 - f1_perClass: 0.7123 - acc: 0.5397     
Epoch 70/100
57867/57867 [==============================] - 1s - loss: 15.0083 - f1_perRow: 0.4110 - f1_perClass: 0.7235 - acc: 0.5291     
Epoch 71/100
57867/57867 [==============================] - 1s - loss: 14.8657 - f1_perRow: 0.4272 - f1_perClass: 0.7206 - acc: 0.5562     
Epoch 72/100
57867/57867 [==============================] - 1s - loss: 14.8186 - f1_perRow: 0.4349 - f1_perClass: 0.7151 - acc: 0.5321     
Epoch 73/100
57867/57867 [==============================] - 1s - loss: 14.6137 - f1_perRow: 0.4378 - f1_perClass: 0.7179 - acc: 0.5288     
Epoch 74/100
57867/57867 [=======

In [31]:
model2.compile(loss=losses,loss_weights=lossWeights, optimizer=keras.optimizers.Adam(lr=8e-5  ), metrics=[f1_perRow,f1_perClass,'acc'])
hist2 = model2.fit(x_lstm_prossed_train2, y_lstm_prossed_train, epochs=300, batch_size=10500, shuffle=True, callbacks=callbacks_list)


Epoch 1/300
57867/57867 [==============================] - 2s - loss: 13.8575 - f1_perRow: 0.4838 - f1_perClass: 0.7169 - acc: 0.7773     
Epoch 2/300
57867/57867 [==============================] - 1s - loss: 13.8491 - f1_perRow: 0.4781 - f1_perClass: 0.7164 - acc: 0.7929     
Epoch 3/300
57867/57867 [==============================] - 1s - loss: 13.8577 - f1_perRow: 0.4780 - f1_perClass: 0.7180 - acc: 0.7817     
Epoch 4/300
57867/57867 [==============================] - 1s - loss: 13.7948 - f1_perRow: 0.4813 - f1_perClass: 0.7165 - acc: 0.7916     
Epoch 5/300
57867/57867 [==============================] - 1s - loss: 13.9823 - f1_perRow: 0.4755 - f1_perClass: 0.7141 - acc: 0.7516     
Epoch 6/300
57867/57867 [==============================] - 1s - loss: 13.8702 - f1_perRow: 0.4823 - f1_perClass: 0.7197 - acc: 0.7405     
Epoch 7/300
57867/57867 [==============================] - 1s - loss: 13.7650 - f1_perRow: 0.4781 - f1_perClass: 0.7179 - acc: 0.7446     
Epoch 8/300
57867/57867 [==

57867/57867 [==============================] - 1s - loss: 13.8340 - f1_perRow: 0.4815 - f1_perClass: 0.7189 - acc: 0.7907     
Epoch 60/300
57867/57867 [==============================] - 1s - loss: 13.8814 - f1_perRow: 0.4833 - f1_perClass: 0.7181 - acc: 0.7789     
Epoch 61/300
57867/57867 [==============================] - 1s - loss: 13.7531 - f1_perRow: 0.4818 - f1_perClass: 0.7179 - acc: 0.7637     
Epoch 62/300
57867/57867 [==============================] - 1s - loss: 13.9827 - f1_perRow: 0.4738 - f1_perClass: 0.7141 - acc: 0.7614     
Epoch 63/300
57867/57867 [==============================] - 1s - loss: 13.8301 - f1_perRow: 0.4785 - f1_perClass: 0.7145 - acc: 0.7324     
Epoch 64/300
57867/57867 [==============================] - 1s - loss: 13.7750 - f1_perRow: 0.4876 - f1_perClass: 0.7205 - acc: 0.7849     
Epoch 65/300
57867/57867 [==============================] - 1s - loss: 13.7935 - f1_perRow: 0.4830 - f1_perClass: 0.7181 - acc: 0.7768     
Epoch 66/300
57867/57867 [=======

57867/57867 [==============================] - 1s - loss: 13.7395 - f1_perRow: 0.4785 - f1_perClass: 0.7201 - acc: 0.7765     
Epoch 118/300
57867/57867 [==============================] - 1s - loss: 13.7728 - f1_perRow: 0.4834 - f1_perClass: 0.7184 - acc: 0.7672     
Epoch 119/300
57867/57867 [==============================] - 1s - loss: 13.8119 - f1_perRow: 0.4794 - f1_perClass: 0.7165 - acc: 0.7466     
Epoch 120/300
57867/57867 [==============================] - 1s - loss: 13.7180 - f1_perRow: 0.4808 - f1_perClass: 0.7180 - acc: 0.7480     
Epoch 121/300
57867/57867 [==============================] - 1s - loss: 13.8033 - f1_perRow: 0.4788 - f1_perClass: 0.7170 - acc: 0.7797     
Epoch 122/300
57867/57867 [==============================] - 1s - loss: 13.7070 - f1_perRow: 0.4756 - f1_perClass: 0.7174 - acc: 0.7773     
Epoch 123/300
57867/57867 [==============================] - 1s - loss: 13.8152 - f1_perRow: 0.4575 - f1_perClass: 0.7106 - acc: 0.7740     
Epoch 124/300
57867/57867 [

57867/57867 [==============================] - 1s - loss: 13.7229 - f1_perRow: 0.4704 - f1_perClass: 0.7112 - acc: 0.7642     
Epoch 176/300
57867/57867 [==============================] - 1s - loss: 13.5327 - f1_perRow: 0.4740 - f1_perClass: 0.7105 - acc: 0.7530     
Epoch 177/300
57867/57867 [==============================] - 1s - loss: 13.6322 - f1_perRow: 0.4694 - f1_perClass: 0.7134 - acc: 0.8043     
Epoch 178/300
57867/57867 [==============================] - 1s - loss: 13.5115 - f1_perRow: 0.4700 - f1_perClass: 0.7096 - acc: 0.8283     
Epoch 179/300
57867/57867 [==============================] - 1s - loss: 13.5763 - f1_perRow: 0.4720 - f1_perClass: 0.7087 - acc: 0.7994     
Epoch 180/300
57867/57867 [==============================] - 1s - loss: 13.9433 - f1_perRow: 0.4551 - f1_perClass: 0.7054 - acc: 0.8004     
Epoch 181/300
57867/57867 [==============================] - 1s - loss: 13.6773 - f1_perRow: 0.4761 - f1_perClass: 0.7143 - acc: 0.8103     
Epoch 182/300
57867/57867 [

57867/57867 [==============================] - 1s - loss: 13.6242 - f1_perRow: 0.4738 - f1_perClass: 0.7105 - acc: 0.7775     
Epoch 234/300
57867/57867 [==============================] - 1s - loss: 13.8852 - f1_perRow: 0.4573 - f1_perClass: 0.7080 - acc: 0.8013     
Epoch 235/300
57867/57867 [==============================] - 1s - loss: 13.6034 - f1_perRow: 0.4772 - f1_perClass: 0.7145 - acc: 0.8145     
Epoch 236/300
57867/57867 [==============================] - 1s - loss: 13.5952 - f1_perRow: 0.4752 - f1_perClass: 0.7137 - acc: 0.8105     
Epoch 237/300
57867/57867 [==============================] - 1s - loss: 13.5306 - f1_perRow: 0.4699 - f1_perClass: 0.7115 - acc: 0.7789     
Epoch 238/300
57867/57867 [==============================] - 1s - loss: 13.7386 - f1_perRow: 0.4665 - f1_perClass: 0.7106 - acc: 0.8000     
Epoch 239/300
57867/57867 [==============================] - 1s - loss: 13.8527 - f1_perRow: 0.4677 - f1_perClass: 0.7080 - acc: 0.8244     
Epoch 240/300
57867/57867 [

57867/57867 [==============================] - 1s - loss: 13.6385 - f1_perRow: 0.4801 - f1_perClass: 0.7154 - acc: 0.7976     
Epoch 292/300
57867/57867 [==============================] - 1s - loss: 13.5728 - f1_perRow: 0.4806 - f1_perClass: 0.7183 - acc: 0.7750     
Epoch 293/300
57867/57867 [==============================] - 1s - loss: 13.5715 - f1_perRow: 0.4802 - f1_perClass: 0.7181 - acc: 0.7694     
Epoch 294/300
57867/57867 [==============================] - 1s - loss: 13.6090 - f1_perRow: 0.4848 - f1_perClass: 0.7160 - acc: 0.7492     
Epoch 295/300
57867/57867 [==============================] - 1s - loss: 13.5882 - f1_perRow: 0.4855 - f1_perClass: 0.7211 - acc: 0.7556     
Epoch 296/300
57867/57867 [==============================] - 1s - loss: 13.7393 - f1_perRow: 0.4711 - f1_perClass: 0.7123 - acc: 0.7517     
Epoch 297/300
57867/57867 [==============================] - 1s - loss: 13.5580 - f1_perRow: 0.4848 - f1_perClass: 0.7166 - acc: 0.7270     
Epoch 298/300
57867/57867 [

In [ ]:
model2.compile(loss=f1_loss, optimizer=keras.optimizers.Adam(lr=8e-5  ), metrics=[f1,'acc'])
hist2 = model2.fit(x_lstm_prossed_train2, y_lstm_prossed_train, epochs=300, batch_size=7500, shuffle=True)

In [ ]:
model2.compile(loss=weighted_categorical_crossentropy(weights=weights), optimizer=keras.optimizers.Adam(lr=8e-5  ), metrics=['accuracy'])

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=8e-5  ), metrics=['accuracy'])

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model2.compile(loss=weighted_categorical_crossentropy(weights=weights), optimizer='adam', metrics=['accuracy'])

In [ ]:
hist2 = model2.fit(x_lstm_prossed_train2, y_lstm_prossed_train, epochs=300, batch_size=3500, shuffle=True)

In [ ]:
model2.save( "LSTM-sigmoid-withRemovedClasses" )

In [ ]:
plt.plot(hist2.history['loss'], c='red')
plt.plot(hist2.history['acc'], c='blue')

In [ ]:
plt.plot(hist2.history['loss'], c='red')
# plt.plot(hist2.history['acc'], c='blue')

In [ ]:
# from keras.models import load_model
# model2=load_model( "LSTM_withSigmoid_LargeData_F1_E100_B500_MSE_False"  
#            )

In [ ]:
for i in range(len(lstm_tests)) :
    print( "==================HOME Case : %s =============" % test_names[i])
    makeReadable( classes=classes, confidance=0.7,data=lstm_tests[i][0],gt=lstm_tests[i][1],model=model2,path=test_names[i],x=lstm_tests[i][0])
    
#     lstm_pred= model2.predict( lstm_tests[i][0])
    

In [ ]:
lstm_pred= model2.predict( lstm_tests[i][0])

In [ ]:
lstm_pred[0]

In [ ]:
len( lstm_pred[0] )

In [37]:
for i in [1] :
# for i in range(len(lstm_tests)) :
    print( "==================HOME Case : %s =============" % test_names[i])
    lstm_pred= model2.predict( lstm_tests[i][0])
    
#     print_info( lstm_tests[i][1], lstm_pred, classes , confidance=0.7)
    print_info( lstm_tests[i][1], lstm_pred, classes , confidance=0.85)
    



==================HOME Case : home_os_final.json =============


/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


                         Class  Accuracy     Recall  Precision   F Score    Count      TP/TN/FP/FN
------------------------------------------------------------------------
                  acceleration     0.998      0.316     0.783     0.450        57 18/19906/5/39
                      activity     1.000      0.632     0.857     0.727        19 12/19947/2/7
                       battery     1.000      0.000       nan     0.000         7 0/19961/0/7
                        button     0.999      0.071     0.500     0.125        14 1/19953/1/13
              colorTemperature     1.000      0.000       nan     0.000         6 0/19962/0/6
                       contact     0.996      0.568     0.901     0.697       176 100/19781/11/76
                         level     0.999      0.222     0.429     0.293        27 6/19933/8/21
                          lock     0.998      0.714     0.439     0.543        35 25/19901/32/10
                        motion     0.981      0.000       nan   

In [ ]:
lstm_pred =model2.predict( x_lstm_prossed_test2)


In [ ]:
print_info(y_lstm_prossed_test, lstm_pred, classes, confidance=0.60)

In [ ]:
lstm_pred =model2.predict( x_lstm_prossed_test)
print_info(y_lstm_prossed_test, lstm_pred, classes, confidance=0.6)

In [ ]:
lstm_pred = model2.predict( x_lstm_prossed_test)
print_info(y_lstm_prossed_test, lstm_pred, classes, confidance=0.6)

In [ ]:
print_info(y_lstm_prossed_train, y_lstm_prossed_train, classes, confidance=0.60)

In [ ]:
[x for x  in lstm_pred  if  np.sum(x) > 0 ]

In [ ]:
model2.save( "LSTM_withSigmoid_LargeData_F%s_E%d_B%d_M%s_%r" %
            (
            FoldID,
                Epoch_count,
                Batch_size,
                Mapper,
                IgnoreEmpty
            ) 
           )

In [ ]:
lstm_pred = model2.predict( x_lstm_prossed_test )

In [ ]:
print_info(y_lstm_prossed_test, lstm_pred, classes)

In [ ]:
# x_train_muhammed,y_train_muhammed, classes = pre_process_raw( x_train, y_train , dim_size, zero_pad=True, normalize=False)
# x_test_muhammed,y_test_muhammed, classes = pre_process_raw( x_test, y_test , dim_size, zero_pad=True, normalize=False)

In [ ]:
lstm_pred = model2.predict( x_lstm_prossed_test )
print_info(y_lstm_prossed_test, lstm_pred, classes)

In [ ]:
lstm_pred = model2.predict( x_lstm_prossed_test )
print_info(y_lstm_prossed_test, lstm_pred, classes)

In [ ]:
pred[:10]

In [ ]:
print_info(y_lstm_prossed_test, pred, classes)

In [ ]:
pred[0]

In [ ]:
len( y_lstm_prossed_train[0] )

In [ ]:
dim_size =160
x_lstm_prossed_train,y_lstm_prossed_train, _ = pre_process_raw( x_train, y_train , dim_size, zero_pad=True, normalize=False,classes=classes)
x_lstm_prossed_test,y_lstm_prossed_test, _ = pre_process_raw( x_test, y_test , dim_size, zero_pad=True, normalize=False,classes=classes)

In [ ]:
len([x  for x  in y_lstm_prossed_test if x[21]==1 or x[20]==1]), len(y_lstm_prossed_test  ) , len([x  for x  in y_lstm_prossed_test if x[21]==1 or x[20]==1])/len(y_lstm_prossed_test  ) *1.0

In [ ]:
[ x for x  in  pred if np.sum(x) > 1]

In [ ]:
len(y_lstm_prossed_test)

In [ ]:
def do_for_raun( pred   ):
    pp = pred
    pp[pp>=0.5] = 1
    pp[pp<0.5] = 0
    return pp

In [ ]:
len([x for x in pred if np.sum( do_for_raun(x) )==0 ])

In [ ]:
len([x for x in pred if  do_for_raun(x)[20] ==1 or do_for_raun(x)[21] ==1 ])

In [ ]:

# np.save("../files/muhammed/x_train.json" , x_train_muhammed)
# np.save("../files/muhammed/y_train.json", )


# np.save( "../files/muhammed/x_train.json", x_train_muhammed )
# np.save("../files/muhammed/y_train.json",  y_train_muhammed )
# np.save( "../files/muhammed/x_test.json",x_test_muhammed )
# np.save( "../files/muhammed/y_test.json",y_test_muhammed )
# np.save( "../files/muhammed/classes.json",  classes )


In [ ]:
len(x_lstm_prossed_test) + len(x_lstm_prossed_train)